In [ ]:
from __future__ import print_function
from ortools.sat.python import cp_model

class RoomsPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, shifts, num_rooms, lectures, num_lectures, num_shifts, sols):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._num_rooms = num_rooms
        self._lectures = lectures
        self._num_lectures = num_lectures
        self._num_shifts = num_shifts
        self._solutions = set(sols)
        self._solution_count = 0
        
    def on_solution_callback(self):
        if self._solution_count in self._solutions:
            print('Solution %i' % self._solution_count)
            for s in range(self._num_shifts):
                print('Shift %i' % s)
                l = 0
                while l < len(self._lectures):
                    is_allocated = False
                    for r in range(self._num_rooms): 
                        if self.Value(self._shifts[(l, s, r)]):
                            is_allocated = True
                            print('  %s occupies room %i' % (self._lectures[l], r))
                            
                    if not is_allocated:
                        print('  %s not allocated ' % (self._lectures[l]))
                    
                    l += 1
                    
            print()
        self._solution_count += 1  
        
    def solution_count(self):
            return self._solution_count
        

def main():
    # Data.
    lectures = ['CMP4205', 'CMP6502', 'ELE3204', 'CIV2205', 'TEL8503', 'ECO0022']
    num_lectures = len(lectures)
    num_rooms = 5
    num_shifts = 3
    all_rooms = range(num_rooms)
    all_shifts = range(num_shifts)
    all_lectures = range(num_lectures)
    
    # Create the model
    model = cp_model.CpModel()
    
    # Creating shift variables
    shifts = {}
    for l in all_lectures:
        for s in all_shifts:
            for r in all_rooms:
                shifts[(l, s, r)] = model.NewBoolVar('shift_l%i s%i r%i' % (l, s, r))
    
    # Each lecture is assigned to exactly one room during a given shift
    for s in all_shifts:
        for r in all_rooms:
            model.Add(sum(shifts[(l, s, r)] for l in all_lectures) == 1)
    
    # Each lecture is allocated at most one room per shift
    for l in all_lectures:
         for s in all_shifts:
             model.Add(sum(shifts[(l, s, r)] for r in all_rooms) <= 1)

    min_rooms_per_lecture = (num_rooms * num_shifts) // num_lectures
    max_rooms_per_lecture = min_rooms_per_lecture + 1

    for l in all_lectures:
        num_lectures_allocated = sum(shifts[(l, s, r)] for s in all_shifts for r in all_rooms)
        model.Add(min_rooms_per_lecture <= num_lectures_allocated)
        model.Add(num_lectures_allocated <= max_rooms_per_lecture)
    
    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    solver.parameters.linearization_level = 0
    
    # Display the first two solutions.
    a_few_solutions = range(2)
    solution_printer = RoomsPartialSolutionPrinter(shifts, num_rooms, lectures, num_lectures, num_shifts, a_few_solutions)
    solver.SearchForAllSolutions(model, solution_printer)
   
     # Statistics.
    print()
    print('Statistics')
    print('  - conflicts       : %i' % solver.NumConflicts())
    print('  - branches        : %i' % solver.NumBranches())
    print('  - wall time       : %f s' % solver.WallTime())
    print('  - solutions found : %i' % solution_printer.solution_count())


if __name__ == '__main__':
    main()

Solution 0
Shift 0
  CMP4205 not allocated 
  CMP6502 occupies room 1
  ELE3204 occupies room 0
  CIV2205 occupies room 2
  TEL8503 occupies room 3
  ECO0022 occupies room 4
Shift 1
  CMP4205 occupies room 4
  CMP6502 occupies room 3
  ELE3204 not allocated 
  CIV2205 occupies room 2
  TEL8503 occupies room 1
  ECO0022 occupies room 0
Shift 2
  CMP4205 occupies room 3
  CMP6502 occupies room 2
  ELE3204 occupies room 1
  CIV2205 occupies room 4
  TEL8503 occupies room 0
  ECO0022 not allocated 

Solution 1
Shift 0
  CMP4205 not allocated 
  CMP6502 occupies room 1
  ELE3204 occupies room 0
  CIV2205 occupies room 2
  TEL8503 occupies room 3
  ECO0022 occupies room 4
Shift 1
  CMP4205 occupies room 4
  CMP6502 not allocated 
  ELE3204 occupies room 3
  CIV2205 occupies room 2
  TEL8503 occupies room 1
  ECO0022 occupies room 0
Shift 2
  CMP4205 occupies room 3
  CMP6502 occupies room 2
  ELE3204 occupies room 1
  CIV2205 occupies room 4
  TEL8503 occupies room 0
  ECO0022 not allocated 